In [1]:
import neuroglancer
import cloudvolume

In [2]:
import numpy as np
from collections import OrderedDict
import webbrowser

In [3]:
neuroglancer.set_static_content_source(url='https://neuromancer-seung-import.appspot.com')

In [21]:
viewer = neuroglancer.Viewer()
# This volume handle can be used to notify the viewer that the data has changed.

with viewer.txn() as s:
    s.layers['segmentation'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1339'
    )
with viewer.txn() as s:
    s.layers[0]._json_data['skeletonRendering']=\
        OrderedDict([('mode2d', 'lines_and_points'), ('mode3d', 'lines')])
#     s.layers[0]._json_data['segments']=unique_segments
print(viewer)

http://127.0.0.1:33939/v/ebb4e3516f339fd6fe70e3217dd12bf97086b05d/


In [14]:
# get th key binding to work
num_actions = 0
def my_action(s):
    global num_actions
    num_actions += 1
    with viewer.config_state.txn() as st:
      st.status_messages['hello'] = ('Got action %d: mouse position = %r' %
                                     (num_actions, s.mouse_voxel_coordinates))
    print('Got my-action %i' % num_actions)
    print('  Mouse position: %s' % (s.mouse_voxel_coordinates,))
    print('  Layer selected values: %s' % (s.selected_values,))
viewer.actions.add('my-action', my_action)
with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer['keyt'] = 'my-action'
    s.status_messages['hello'] = 'Welcome to this example'

Got my-action
  Mouse position: [176.5 302.  271. ]
  Layer selected values: Map({"segmentation": 362})
Got my-action 2
  Mouse position: [176.5 302.  271. ]
  Layer selected values: Map({"segmentation": 362})


In [23]:
atlas_fake_dict = {362:'Neocortex'} 
# get the message to print out the value in the fake atlas dictionary for the segment selected in the viewer
num_actions = 0
def my_action2(s):
    global num_actions
    num_actions += 1
    with viewer.config_state.txn() as st:
        region_id = s.selected_values['segmentation']
        region_name = atlas_fake_dict.get(region_id)
        st.status_messages['hello'] = ('%i:%s' %
                                     (region_id, region_name))
    print('Got my-action %i' % num_actions)
#     print('  Mouse position: %s' % (s.mouse_voxel_coordinates,))
    print('  Layer selected values: %s' % (s.selected_values,))
viewer.actions.add('my-action2', my_action2)
with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer['keyp'] = 'my-action2'
    s.status_messages['hello'] = 'Welcome to the segment metadata example. Press p to see region name when hovering.'



Got my-action 1
  Layer selected values: Map({"segmentation": 362})
Got my-action 2
  Layer selected values: Map({"segmentation": 362})
